# MLFlow-இல் Phi-3 மாதிரியை இயக்குதல்


---------
இந்த நோட்புக் Phi-3 mini 4K Instruct என்ற உதாரணத்தின் மூலம் ஒரு தனிப்பயன் Python ராப்பரைப் பயன்படுத்துவதை விவரிக்கிறது.


In [1]:
# Importing required packages
import mlflow
from mlflow.models import infer_signature
import onnxruntime_genai as og

### Phi-3 Mini 4K மாடலுக்கான தனிப்பயன் Python வகுப்பை வரையறுத்தல்


In [2]:
# Defining custom PythonModel class
class Phi3Model(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        # Retrieving model from the artifacts
        model_path = context.artifacts["phi3-mini-onnx"]
        model_options = {
             "max_length": 300,
             "temperature": 0.2,         
        }
    
        # Defining the model
        self.phi3_model = og.Model(model_path)
        self.params = og.GeneratorParams(self.phi3_model)
        self.params.set_search_options(**model_options)
        
        # Defining the tokenizer
        self.tokenizer = og.Tokenizer(self.phi3_model)

    def predict(self, context, model_input):
        # Retrieving prompt from the input
        prompt = model_input["prompt"][0]
        self.params.input_ids = self.tokenizer.encode(prompt)

        # Generating the model's response
        response = self.phi3_model.generate(self.params)

        return self.tokenizer.decode(response[0][len(self.params.input_ids):])

### MLFlow அர்டிஃபாக்ட் உருவாக்குதல்


In [3]:
# Generating the MLflow model with the custom Python model
input_example = {"prompt": "<|system|>You are a helpful AI assistant.<|end|><|user|>What is the capital of Spain?<|end|><|assistant|>"}
artifact_path = "phi3_mlflow_model"

with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        artifact_path = artifact_path,
        python_model = Phi3Model(),
        artifacts = {
            "phi3-mini-onnx": "cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4",
        },
        input_example = input_example,
        signature = infer_signature(input_example, ["Run"]),
        extra_pip_requirements = ["torch", "onnxruntime_genai", "numpy"],
    )

2024/06/18 14:17:34 INFO mlflow.models.utils: We convert input dictionaries to pandas DataFrames such that each key represents a column, collectively constituting a single row of data. If you would like to save data as multiple rows, please convert your data to a pandas DataFrame before passing to input_example.


### Phi-3 ஐ MLFlow மாடலாக இயக்குதல்


In [4]:
# Loading Phi-3 MLFlow model
loaded_model = mlflow.pyfunc.load_model(
    model_uri = model_info.model_uri
)

In [5]:
# Checking MLFlow model's signature
loaded_model.metadata.signature

inputs: 
  ['prompt': string (required)]
outputs: 
  [string (required)]
params: 
  None

In [6]:
# Testing the loaded model
response = loaded_model.predict(
    {"prompt": "<|system|>You are a stand-up comedian.<|end|><|user|>Tell me a joke about atom<|end|><|assistant|>",}
)
print(response)

 Alright, here's a little atom-related joke for you!

Why don't electrons ever play hide and seek with protons?

Because good luck finding them when they're always "sharing" their electrons!

Remember, this is all in good fun, and we're just having a little atomic-level humor!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
மறுப்பு:
இந்த ஆவணம் Co-op Translator (https://github.com/Azure/co-op-translator) என்ற எஐ மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிப்போம் என்றாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கலாம் என்பதை தயவுசெய்து கவனிக்கவும். மூல ஆவணம் அதன் இயல்பான மொழியிலேயே அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பை பரிந்துரைக்கிறோம். இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்தவொரு தவறான புரிதல்களுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பேற்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
